In [48]:
import cv2
import os
import pandas as pd
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [49]:
#Informacion
dir_img=r'C:\Users\kvnsg\Desktop\ML\Proyecto\Images'
list_dirimg=[]
clases=[]
etiquetas=[]

In [50]:
##Obtener clases
clases=os.listdir(dir_img)
print(clases)

['fresh_apple', 'fresh_banana', 'fresh_bitter_gourd', 'fresh_capsicum', 'fresh_orange', 'fresh_tomato', 'stale_apple', 'stale_banana', 'stale_bitter_gourd', 'stale_capsicum', 'stale_orange', 'stale_tomato']


In [51]:
##Obtener un DF con las rutas de las imagenes y su correspondiente etiqueta
def loadImg(clases,dir_img):    
    for i in clases:
        dir_clase=os.path.join(dir_img,i)#Obtener la ruta de cada clase
        if os.path.isdir(dir_clase):     #Pregunta si el directorio existe
            nombreImg=os.listdir(dir_clase)  
            for j in nombreImg:
                rutaImg=os.path.join(dir_clase,j)#Obtener la ruta de cada imagen
                list_dirimg.append(rutaImg)      #Agrega un ítem al final de la lista.     
                etiquetas.append(i) 
    Rseries= pd.Series(list_dirimg, name='ruta_imagenes') #Vector con datos indexados
    Eseries=pd.Series(etiquetas, name='Etiqueta')    
    df=pd.concat([Rseries, Eseries], axis=1)
    return df

In [52]:
df=loadImg(clases,dir_img)

In [53]:
df

,ruta_imagenes,Etiqueta
0,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_apple
1,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_apple
2,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_apple
3,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_apple
4,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_apple
...,...,...
14677,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_tomato
14678,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_tomato
14679,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_tomato
14680,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_tomato


In [54]:
iFB=df.index[df['Etiqueta'] == 'fresh_bitter_gourd']
iSB=df.index[df['Etiqueta'] == 'stale_bitter_gourd']

In [55]:
df_FB=df.loc[iFB.values.tolist(), ['ruta_imagenes','Etiqueta']]
df_SB=df.loc[iSB.values.tolist(), ['ruta_imagenes','Etiqueta']]

In [56]:
df_FB.head()

,ruta_imagenes,Etiqueta
3274,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_bitter_gourd
3275,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_bitter_gourd
3276,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_bitter_gourd
3277,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_bitter_gourd
3278,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\fres...,fresh_bitter_gourd


In [57]:
df_SB.head()

,ruta_imagenes,Etiqueta
10847,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_bitter_gourd
10848,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_bitter_gourd
10849,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_bitter_gourd
10850,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_bitter_gourd
10851,C:\Users\kvnsg\Desktop\ML\Proyecto\Images\stal...,stale_bitter_gourd


In [58]:
#Configuración
FSB_datagen=ImageDataGenerator(
    shear_range=0.3, #Inclina las imágenes
    zoom_range=0.3,  #Hacer zoom a las imágenes
    horizontal_flip=True #Invierta la imágenen de manera horizontal
)

In [59]:
#Aplicación
def aumentoDatos(df,datagen,bs,x):
    imGen=datagen.flow_from_dataframe( 
        df,
        directory=None,
        x_col=x, 
        y_col='Etiqueta',
        weight_col=None,
        target_size=(128,128),
        color_mode='rgb',
        classes=None,
        class_mode='categorical',
        batch_size=bs)
    return imGen

In [60]:
gen_FB=aumentoDatos(df_FB,FSB_datagen,1,'ruta_imagenes')
gen_SB=aumentoDatos(df_SB,FSB_datagen,1,'ruta_imagenes')

Found 327 validated image filenames belonging to 1 classes.
Found 357 validated image filenames belonging to 1 classes.


In [75]:
def guardarImg(datagen, bs,ruta):
    datagen.reset()
    for i in range(int(len(datagen)/bs)):
        img, label = next(datagen)
        imgcv2=img[0][:,:,::-1]
        cv2.imwrite('C:/Users/kvnsg/Desktop/ML/Proyecto/Balanceo/'+ruta+'-image-'+str(i)+'.png',imgcv2) 

In [76]:
rutaFB='fresh_bitter_gourd/fb'
rutaSB='stale_bitter_gourd/sb'

In [77]:
guardarImg(gen_FB, 1,rutaFB)

In [78]:
guardarImg(gen_SB, 1,rutaSB)